In [2]:
import tools
import param
import cartesian
import racks
import calibration
import samples
import json
import importlib
import random as rnd
import time
import low_level_comm as llc

In [3]:
def dropTipToWaste(tool, x_slot, y_slot, z):
    tool.robot.move(z=z)
    x, y = param.calcSquareSlotCenterFromVertices(x_slot, y_slot)
    tool.robot.move(x=x, y=y)
    tool.dropTip()

In [4]:
llc.listSerialPorts()

['COM3', 'COM6', 'COM10', 'COM14']

In [5]:
ar = cartesian.arnie(cartesian_port="COM10", docker_port="COM6")

In [6]:
ar.home()

In [7]:
stp = tools.stationary_touch_probe(ar)

ERROR:root:matchPortsWithDevices(): could not execute finiding device in list.
ERROR:root:matchPortsWithDevices(): Provided patterns_list: ['stationary touch probe']
ERROR:root:None


In [8]:
p1000_tip_rack = racks.consumables(rack_name='p1000_tips_1', rack_type='p1000_tips')

In [9]:
ep1 = racks.rack(rack_name='eppendorf_1', rack_type='eppendorf')
r50 = racks.rack(rack_name='50mL_rack_1', rack_type='50mL')

In [19]:
sample_list = [samples.sample(sample_name='t_ep_'+str(x), sample_type='eppendorf_tube') for x in range(5)]
[x[0].place(ep1, x[1], 3) for x in zip(sample_list, range(5))]

[None, None, None, None, None]

In [20]:
ethanol = samples.sample(sample_name='ethanol', sample_type='50mL_tube')
ethanol.place(r50, 2, 0)
ethanol.setVolume(50000)

In [12]:
p1000 = tools.pipettor(robot=ar, tool_name='p1000_tool')

In [13]:
xt, yt, zt = calibration.calibrateTool(p1000, stp)

In [ ]:
# TODO: Calibrate uptake heights. Now it is too low
xn, yn = p1000_tip_rack.getNextConsumable()
p1000.pickUpTip(p1000_tip_rack, xn, yn, raise_z=400)
ethanol_vol_list = [200 for x in sample_list]
timestamp = time.time()
p1000.distributeLiquid(ethanol, sample_list, ethanol_vol_list, raise_z=300)
dropTipToWaste(tool=p1000, x_slot=5, y_slot=3, z=300)

In [17]:
p1000.movePlunger(0)

In [18]:
dropTipToWaste(tool=p1000, x_slot=5, y_slot=3, z=300)